In [27]:
import pickle
import os
import pandas as pd

In [28]:
dataset_dir = "/mnt/LS226/NAS_FLAME/palaashg/palaashg/Dataset_OCR/Dataset/"
df = pd.read_csv("/mnt/LS226/NAS_FLAME/palaashg/palaashg/Dataset_OCR/answers.csv")
mcq_df = df[df["Type"] == "MCQs"]
int_df = df[df["Type"] == "Integer"]

In [29]:
with open(os.path.join(dataset_dir, "qwen_vl_out.pkl"), "rb") as f:
    llm_out = pickle.load(f)

In [30]:
mcq_df

,Exam Name,Year,Subject (if any),Paper Number (if any),Type,Image Name,Answer Type,Answers,Latex
210,AMC 12,2010,-,12A,MCQs,1.png,-,C,What is $(20-(2010-201))+(2010-(201-20))$ ?\n(...
211,AMC 12,2010,-,12A,MCQs,2.png,-,A,A ferry boat shuttles tourists to an island ev...
212,AMC 12,2010,-,12A,MCQs,3.png,-,E,"Rectangle $A B C D$, pictured below, shares $5..."
213,AMC 12,2010,-,12A,MCQs,4.png,-,D,"If $x<0$, then which of the following must be ..."
214,AMC 12,2010,-,12A,MCQs,5.png,-,C,"Halfway through a 100-shot archery tournament,..."
...,...,...,...,...,...,...,...,...,...
1705,AMC 10,2024,-,10B,MCQs,21.png,-,C,NaN
1706,AMC 10,2024,-,10B,MCQs,22.png,-,A,NaN
1707,AMC 10,2024,-,10B,MCQs,23.png,-,B,NaN
1708,AMC 10,2024,-,10B,MCQs,24.png,-,E,NaN


In [31]:
llm_out

{'mcq': [{'img_file_path': '/mnt/LS226/NAS_FLAME/palaashg/palaashg/Dataset_OCR/Dataset/AMC 12/2010/12A/MCQs/1.png',
   'answer': 'C',
   'preprocessing_results': {'preprocessed': array([[255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           ...,
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
    'question': array([[255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           ...,
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
    'options': array([[255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           [255, 255, 255, ..., 255, 255, 255],
           ...,
    

# Evaluation - Integer Based Questions

In [33]:
llm_out_int = llm_out["int"]
llm_out_mcq = llm_out["mcq"]

correct = 0
total = 0

for sample in llm_out_int:
    gen_response = sample["gen_response"].split("\\boxed{")
    if (len(gen_response) == 1):
        continue
    
    gen_ans = gen_response[-1].split("}")
    if (len(gen_ans) == 1):
        continue
    
    gen_ans = str(gen_ans[0]).strip()
        
    if (gen_ans == str(sample["answer"]).strip()):
        correct += 1
    
    total += 1

acc_int = correct / total

#MCQs
correct = 0
total = 0

for idx, sample in enumerate(llm_out_mcq):
    gen_response = sample["gen_response"].split("\\boxed{")
    row = mcq_df.iloc[idx, :]
    if (len(gen_response) == 1):
        continue
    
    gen_ans = gen_response[-1].split("}")
    if (len(gen_ans) == 1):
        continue
    
    gen_ans = str(gen_ans[0]).strip() #This is a number not the option, need to get the option number for accuracy computation
    
    correct_option = sample["answer"]
    correct_ans_val = str(row["Latex"].split(f"({correct_option})")[-1]).strip()
    correct_ans_val = correct_ans_val.split("\n")[0]
    
    if (gen_ans == correct_ans_val):
        correct += 1

    total += 1
acc_mcq = correct / total

print(f"Accuracy on Integer-Based Questions: {acc_int:.4f}")
print(f"Accuracy on MCQs: {acc_mcq:.4f}")

Accuracy on Integer-Based Questions: 0.0000
Accuracy on MCQs: 0.3333
